In [20]:
import veloxchem as vlx
import adcc
from mpi4py import MPI
import sys
import numpy
from scipy.sparse import linalg
from veloxchem.qqscheme import get_qq_scheme
from veloxchem.veloxchemlib import fockmat

numpy.set_printoptions(precision=7, suppress=True)

### Prepare a VeloxChem SCF calculation

In [2]:
# Input settings
molecule_string = """
    O 0 0 0
    H 0 0 1.795239827225189
    H 1.693194615993441 0 -0.599043184453037"""

#molecule_string = """
#    H 0 0 0
#    H 0 0 1.795239827225189
#    """

basis_set_label = 'STO-3G'
scf_settings = {'conv_thresh': 1.0e-6}
method_settings = {} #{'xcfun': 'b3lyp', 'grid_level': 4}
rsp_settings = {'conv_thresh': 1.0e-4, 'nstates': 3}

In [3]:
# Communicator and output stream
comm = MPI.COMM_WORLD
ostream = vlx.OutputStream(sys.stdout)

In [4]:
# Molecule and basis set
molecule = vlx.Molecule.read_str(molecule_string, units='au')
basis = vlx.MolecularBasis.read(molecule, basis_set_label)

ostream.print_block(molecule.get_string())
ostream.print_block(basis.get_string('Atomic Basis', molecule))
ostream.flush()

                                              Molecular Geometry (Angstroms)                                              
                                                                                                                          
                          Atom         Coordinate X          Coordinate Y          Coordinate Z                           
                                                                                                                          
                           O           0.000000000000        0.000000000000        0.000000000000                         
                           H           0.000000000000        0.000000000000        0.950000004704                         
                           H           0.896000004436        0.000000000000       -0.317000001569                         
                                                                                                                          
                

In [5]:
# SCF
scfdrv = vlx.ScfRestrictedDriver(comm, ostream)
scfdrv.update_settings(scf_settings, method_settings)
scfdrv.compute(molecule, basis)

                                                                                                                          
                                            Self Consistent Field Driver Setup                                            
                                                                                                                          
                   Wave Function Model             : Spin-Restricted Hartree-Fock                                         
                   Initial Guess Model             : Superposition of Atomic Densities                                    
                   Convergence Accelerator         : Two Level Direct Inversion of Iterative Subspace                     
                   Max. Number of Iterations       : 50                                                                   
                   Max. Number of Error Vectors    : 10                                                                   
                

In [6]:
# SCF first-order properties
scf_prop = vlx.ScfFirstOrderProperties(comm, ostream)
scf_prop.compute(molecule, basis, scfdrv.scf_tensors)
scf_prop.print_properties(molecule)

                                                                                                                          
                                                Ground-State Dipole Moment                                                
                                               ----------------------------                                               
                                                                                                                          
                                   X   :         0.546314 a.u.         1.388593 Debye                                     
                                   Y   :        -0.000000 a.u.        -0.000000 Debye                                     
                                   Z   :         0.386302 a.u.         0.981881 Debye                                     
                                 Total :         0.669095 a.u.         1.700671 Debye                                     
                

### Excited-State Calculation via CIS (TDHF/TDA)

In [7]:
# TDHF/TDA, i.e. CIS
tda_drv = vlx.TDAExciDriver(comm, ostream)
tda_drv.update_settings(rsp_settings, method_settings)
tda_results = tda_drv.compute(molecule, basis, scfdrv.scf_tensors)
tda_drv.ostream.flush()

                                                                                                                          
                                                     TDA Driver Setup                                                     
                                                                                                                          
                               Number of States                : 3                                                        
                               Max. Number of Iterations       : 150                                                      
                               Convergence Threshold           : 1.0e-04                                                  
                               ERI Screening Scheme            : Cauchy Schwarz + Density                                 
                               ERI Screening Threshold         : 1.0e-15                                                  
                

In [8]:
# Get MO coefficients

nocc = molecule.number_of_alpha_electrons()
mo = scfdrv.scf_tensors['C'] # MO coefficients
mo_occ = mo[:, :nocc]        # occupied
mo_vir = mo[:, nocc:]        # virtual

nocc = mo_occ.shape[1]
nvir = mo_vir.shape[1]

In [9]:
# Get the overlap and Fock matrices
print(scfdrv.scf_tensors.keys())
fock = scfdrv.scf_tensors['F'][0]
ovlp = scfdrv.scf_tensors['S']
# Transform them from AO to MO basis
# print(numpy.matmul(mo.T, numpy.matmul(ovlp, mo)))
# print(numpy.matmul(mo.T, numpy.matmul(fock, mo)))

dict_keys(['C', 'E', 'S', 'D', 'F'])


In [10]:
# Get the CIS eigenvalues and eigenvectors
tda_eig_vals = tda_results["eigenvalues"]
tda_eig_vecs = tda_results["eigenvectors"]
tda_size = tda_eig_vecs[:,0].shape
tda_eig_vec=tda_eig_vecs[:,0].copy()
tda_eig_vec_as_mat = tda_eig_vec.reshape(nocc, nvir) #/ numpy.sqrt(2.0)

### Calculate one- and two-particle density matrices

\begin{align}
    \gamma_{ij} &= - \sum_{a} x_{ia} x_{ja} \\
    \gamma_{ab} &= + \sum_{i} x_{ia} x_{ib} \\
    \Gamma_{iajb} &= - x_{ib} x_{ja}
\end{align}

In [11]:
# VeloxChem unrelaxed one-particle density matrix and two-particle density matrix

vlx_dm_oo = -numpy.einsum('ia,ja->ij', tda_eig_vec_as_mat, tda_eig_vec_as_mat)
vlx_dm_vv = numpy.einsum('ia,ib->ab', tda_eig_vec_as_mat, tda_eig_vec_as_mat)
vlx_DM_ovov = -numpy.einsum('ib,ja->iajb', tda_eig_vec_as_mat, tda_eig_vec_as_mat)

# print(vlx_dm_oo)

### Transform the one-particle density matrices from the MO to the AO basis

\begin{equation}
    \gamma_{\mu \nu} = \sum_{pq} C_{\mu p} \gamma_{pq} C_{\nu q}
\end{equation}

In [12]:
# Transform the one-particle density matrices from MO to AO basis
vlx_dm_oo_in_ao = numpy.matmul(mo_occ, numpy.matmul(vlx_dm_oo, mo_occ.T))
vlx_dm_vv_in_ao = numpy.matmul(mo_vir, numpy.matmul(vlx_dm_vv, mo_vir.T))
vlx_dm_ao = vlx_dm_oo_in_ao + vlx_dm_vv_in_ao

# print("VLX:")
# print(vlx_dm_ao)

#### Transform the excitation vectors to the AO basis
\begin{equation}
    x_{\mu \nu} = \sum_{ia} C_{\mu i} x_{ia} C_{\nu a}
\end{equation}

In [13]:
# Transform the transition density matrix (the excitation vectors) from MO to AO basis
# (will be needed for the two-particle density matrix)
tda_eig_vec_ao = numpy.matmul(mo_occ, numpy.matmul(tda_eig_vec_as_mat, mo_vir.T))
# print(tda_eig_vec_ao)

The MO to AO transformation of the contribution of the two-particle density matrix with the ERI tensor is given as
\begin{align}
\begin{split}
    - \sum_{\sigma \sigma'} \sum_{ijab} x_{i_{\sigma} b_{\sigma}} x_{j_{\sigma'} a_{\sigma'}}
        \langle i_{\sigma} a_{\sigma'} || j_{\sigma'} b_{\sigma} \rangle
    &= - \sum_{ijab} x_{ib} x_{ja} [ 2 (ij|ab) - 4 (ib|aj) ] \\
    &= + \sum_{ijab} x_{ib} x_{ja} [ 4 (ib|aj) - 2 (ij|ab)  ] \\
    &= \sum_{ijab} x_{ib} x_{ja} \sum_{\mu \nu \theta \varphi}
        C_{\mu i} C_{\nu b} C_{\theta a} C_{\varphi j}  [ 4 (\mu \nu | \theta \varphi)
        - 2 (\mu \varphi | \theta \nu) ] \\
    &= \sum_{\mu \nu \theta \varphi} [ 4 (\mu \nu | \theta \varphi)
        - 2 (\mu \varphi | \theta \nu) ]
        \sum_{ib} C_{\mu i} x_{ib} C_{\nu b} \sum_{ja} C_{\varphi j} x_{ja} C_{\theta a} \\
    &= \sum_{\mu \nu \theta \varphi} x_{\mu \nu} x_{\varphi \theta} [ 4 (\mu \nu | \theta \varphi)
        - 2 (\mu \varphi | \theta \nu) ]       \\
    &= \sum_{\mu \nu} x_{\mu \nu} \sum_{\theta \varphi} x_{\varphi \theta} [ 4 (\mu \nu | \theta \varphi)
        - 2 (\mu \varphi | \theta \nu) ]
\end{split}
\end{align}

# Orbital response

\begin{equation}
   -\sum_{pq} \lambda_{pq} \big( [pq|ai] - [pi|aq] \big) + \lambda_{ia} ( \varepsilon_i - \varepsilon_a ) = 
   R_{ia}
   %R^{\text{CIS}}_{ia}
\end{equation}

## Right-hand side

### 1. Contributions from the 1PDMs
\begin{align}
F^\gamma_{ia} &= \sum_{\theta,\varphi}C^\alpha_{\varphi i}C^\alpha_{\theta a}\sum_{\mu,\nu,\sigma}\gamma^\sigma_{\mu\nu}(\mu\nu|\theta \varphi)-\sum_{\theta,\varphi}C^\alpha_{\varphi i}C^\alpha_{\theta a}\sum_{\mu,\nu}\gamma^{\alpha}_{\mu\nu}(\mu\varphi|\theta\nu)\\
&= \sum_{\theta,\varphi}C^\alpha_{\varphi i}C^\alpha_{\theta a} 2F^{1,\alpha}_{\varphi\theta}-\sum_{\theta,\varphi}C^\alpha_{\varphi i}C^\alpha_{\theta a}F^{2,\alpha}_{\varphi\theta}\\
&= \sum_{\theta,\varphi}C^\alpha_{\varphi i}C^\alpha_{\theta a}F_{\varphi\theta}
\end{align}

### 2. Contributions from the 2PDMs

\begin{align}
F^\Gamma_{ia} &= ~~ \sum_{\mu,\zeta}C^\alpha_{\mu i}C^\alpha_{\zeta a} \Big[\sum_{\rho,\varphi}S_{\rho\zeta}x^\alpha_{\varphi\rho}\sum_{\nu,\theta}x^\alpha_{\theta\nu}(\mu\nu|\theta\varphi)-\sum_{\rho,\varphi}S_{\rho\zeta}x^\alpha_{\varphi\rho}\sum_{\theta,\nu,\sigma} x^\sigma_{\theta\nu}(\mu\varphi|\theta\nu) \Big] \\
&~~~ -\sum_{\mu,\zeta}C^\alpha_{\mu i}C^\alpha_{\zeta a} \Big[ \sum_{\rho,\varphi}S_{\mu\rho}x^\alpha_{\rho\varphi}\sum_{\nu,\theta}x^\alpha_{\nu\theta}(\zeta\nu|\theta\varphi)-\sum_{\rho,\varphi} S_{\mu\rho}x^\alpha_{\rho\varphi}\sum_{\theta,\nu,\sigma}x^\sigma_{\nu\theta}(\zeta\varphi|\theta\nu) \Big] \\
&= - \sum_{\mu,\zeta}C^\alpha_{\mu i}C^\alpha_{\zeta a} \sum_{\rho,\varphi}S_{\rho\zeta}x^\alpha_{\varphi\rho}\Big[ 2\sum_{\theta,\nu} x^\alpha_{\theta\nu}(\mu\varphi|\theta\nu)- \sum_{\nu,\theta}x^\alpha_{\theta\nu}(\mu\nu|\theta\varphi) \Big] \\
&~~~~ +\sum_{\mu,\zeta}C^\alpha_{\mu i}C^\alpha_{\zeta a} \sum_{\rho,\varphi}S_{\mu\rho}x^\alpha_{\rho\varphi}\Big[ 2\sum_{\theta,\nu} x^\alpha_{\nu\theta}(\zeta\varphi|\theta\nu)- \sum_{\nu,\theta}x^\alpha_{\nu\theta}(\zeta\nu|\theta\varphi) \Big] \\
&= - \sum_{\mu,\zeta}C^\alpha_{\mu i}C^\alpha_{\zeta a} \sum_{\rho,\varphi}S_{\rho\zeta}x^\alpha_{\varphi\rho}\Big[ \sum_{\theta,\nu} x^\alpha_{\theta\nu} \big[ 2 (\mu\varphi|\theta\nu)- (\mu\nu|\theta\varphi) \big] \Big] \\
&~~~~ +\sum_{\mu,\zeta}C^\alpha_{\mu i}C^\alpha_{\zeta a} \sum_{\rho,\varphi}S_{\mu\rho}x^\alpha_{\rho\varphi}\Big[ \sum_{\theta,\nu} x^\alpha_{\nu\theta} \big[ 2(\zeta\varphi|\theta\nu)- (\zeta\nu|\theta\varphi) \big] \Big]
\end{align}

In [14]:
# Calculate two different parts and transform the sum to MO
nao = mo.shape[0]

pqrs = numpy.zeros((nao, nao, nao, nao))
eri_drv_pqrs = vlx.ElectronRepulsionIntegralsDriver(comm)
eri_drv_pqrs.compute_in_mem(molecule, basis, pqrs)

F1_vlx = numpy.einsum('mn,mntp->pt', vlx_dm_ao, pqrs)
F2_vlx = numpy.einsum('mn,mptn->pt', vlx_dm_ao, pqrs)
# The density matrices already have a factor of 2 (for alpha and beta spin)
# hence a factor of 0.5 where the other spin part actually vanishes
F_1pdm_vlx = F1_vlx - 0.5*F2_vlx
vlx_1pdm_rhs = numpy.einsum('pi,pa->ia', mo_occ, numpy.einsum('ta,pt->pa', mo_vir, F_1pdm_vlx))
print(vlx_1pdm_rhs)

[[-0.018333   0.0000249]
 [-0.0827309 -0.0000336]
 [ 0.0004581 -0.1069934]
 [-0.0907946  0.0001547]
 [-0.        -0.       ]]


# Start -- Orbital Response 

In [15]:
# Set up ERI driver:
nao = mo.shape[0]
#Probably these will be properties of our OrbitalResponseSolver?
qq_type = 'QQ_DEN'
eri_thresh = 1.0e-15
eri_drv = vlx.ElectronRepulsionIntegralsDriver(comm)
screening = eri_drv.compute(get_qq_scheme(qq_type),
                            eri_thresh, molecule, basis)
# print(dir(screening))
# print(screening.get_screener(7))


In [24]:
# Use AODensityMatrix object to compute a Fock-like matrix
# expect something like F_{\varphi\theta}, equation above
# see linearsolver.py; we have to create a linear solver object:
# 
# from veloxchem.linearsolver import LinearSolver
from veloxchem.veloxchemlib import AOFockMatrix
from veloxchem.veloxchemlib import AODensityMatrix
from veloxchem.veloxchemlib import denmat

#Create all AODensityMatrices needed for the RHS
#1PDM contribution:
ao_density_1pdm = AODensityMatrix([vlx_dm_ao], denmat.rest)

#2PDM contribution
ao_density_2pdm_1 = AODensityMatrix([tda_eig_vec_ao.T], denmat.rest)
ao_density_2pdm_2 = AODensityMatrix([tda_eig_vec_ao], denmat.rest)

#We only use one of the terms (i.e. ao_density_2pdm_2 -- coresponding to tda_eig_vec_ao)
#because we can get the other as its transpose
ao_density_RHS_all = AODensityMatrix([vlx_dm_ao, tda_eig_vec_ao], denmat.rest)

print("Single terms:")
print("1PDM")
print(ao_density_1pdm)
print()
print("2PDM:")
print(ao_density_2pdm_1)
print()
print(ao_density_2pdm_2)

print("ALL RHS:")
print(ao_density_RHS_all)

Single terms:
1PDM
Density Type: Restricted Density Matrix
[Dimension 7 x 7]
     0.01838912    -0.12316116     0.10921152     0.10970854     0.00000000    -0.05566002    -0.07927709
    -0.12316116     0.82487198    -0.73144421    -0.73477300    -0.00000000     0.37278303     0.53095833
     0.10921152    -0.73144421     0.64859839     0.65155015     0.00000000    -0.33056037    -0.47082021
     0.10970854    -0.73477300     0.65155015     0.65451534     0.00000000    -0.33206475    -0.47296291
     0.00000000    -0.00000000     0.00000000     0.00000000    -1.00000000    -0.00000000    -0.00000000
    -0.05566002     0.37278303    -0.33056037    -0.33206475    -0.00000000     0.16847122     0.23995512
    -0.07927709     0.53095833    -0.47082021    -0.47296291    -0.00000000     0.23995512     0.34177031


2PDM:
Density Type: Restricted Density Matrix
[Dimension 7 x 7]
    -0.00000000     0.00000000    -0.00000000    -0.00000000     0.13560650     0.00000000     0.00000000
     0.00

In [25]:
fock_rhs_1pdm = AOFockMatrix(ao_density_1pdm)
fock_rhs_2pdm_1 = AOFockMatrix(ao_density_2pdm_1)
fock_rhs_2pdm_2 = AOFockMatrix(ao_density_2pdm_2)

fock_RHS_all = AOFockMatrix(ao_density_RHS_all)
fock_flag = fockmat.rgenjk
for i in range(fock_RHS_all.number_of_fock_matrices()):
    fock_RHS_all.set_fock_type(fock_flag, i)
    
fock_rhs_2pdm_1.set_fock_type(fock_flag, 0)
fock_rhs_2pdm_2.set_fock_type(fock_flag, 0)

AO_rhs_1pdm = eri_drv.compute(fock_rhs_1pdm, ao_density_1pdm, molecule, basis, screening) 
AO_rhs_2pdm_1 = eri_drv.compute(fock_rhs_2pdm_1, ao_density_2pdm_1, molecule, basis, screening) 
AO_rhs_2pdm_2 = eri_drv.compute(fock_rhs_2pdm_2, ao_density_2pdm_2, molecule, basis, screening) 

AO_RHS_all = eri_drv.compute(fock_RHS_all, ao_density_RHS_all, molecule, basis, screening)

print("Fock matrices, single terms:")
print("Fock-1PDM")
print(fock_rhs_1pdm)
print()
print("Fock-2PDM:")
print(fock_rhs_2pdm_1)
print()
print(fock_rhs_2pdm_2)

print("Fock-ALL RHS:")
print(fock_RHS_all)



Fock matrices, single terms:
Fock-1PDM
Fock Type: Restricted 2J + K Matrix
Density Identifier: 0
[Dimension 7 x 7]
    -0.65125674    -0.12604156    -0.02685659    -0.02680190    -0.00000000     0.02126198     0.03030670
    -0.12604156    -0.34263392    -0.04608774    -0.04532876     0.00000000     0.05187415     0.07412870
    -0.02685659    -0.04608774    -0.03228598    -0.03918027    -0.00000000     0.01156119     0.08146541
    -0.02680190    -0.04532876    -0.03918027    -0.03060907     0.00000000     0.07221715     0.03922898
    -0.00000000     0.00000000    -0.00000000     0.00000000     0.25092946    -0.00000000     0.00000000
     0.02126198     0.05187415     0.01156119     0.07221715    -0.00000000    -0.39437864    -0.06132726
     0.03030670     0.07412870     0.08146541     0.03922898     0.00000000    -0.06132726    -0.43834986


Fock-2PDM:
Fock Type: Restricted general 2J + K Matrix
Density Identifier: 0
[Dimension 7 x 7]
    -0.00000000    -0.00000000     0.00000000 

In [26]:
#Transform the 1PDM contribution to the RHS to MO basis:
MO_rhs_1pdm = numpy.einsum('pi,pa->ia', mo_occ,
                    numpy.einsum('ta,pt->pa', mo_vir, 0.5*fock_RHS_all.alpha_to_numpy(0)))

print("\nIn MO basis, 1PDM contribution to the RHS, using one-shot Fock-build?:")
print(MO_rhs_1pdm)
print()
print("Original, for comparison:")
print(vlx_1pdm_rhs)


In MO basis, 1PDM contribution to the RHS, using one-shot Fock-build?:
[[-0.018333   0.0000249]
 [-0.0827309 -0.0000336]
 [ 0.0004581 -0.1069934]
 [-0.0907946  0.0001547]
 [-0.        -0.       ]]

Original, for comparison:
[[-0.018333   0.0000249]
 [-0.0827309 -0.0000336]
 [ 0.0004581 -0.1069934]
 [-0.0907946  0.0001547]
 [-0.        -0.       ]]


In [31]:
#Contract with excitation vector, overlap integrals and transform the 2PDM contribution to the RHS to MO basis:
#print(fock_RHS_all.alpha_to_numpy(1))

#LR_2pdm_F1 =  numpy.einsum('rz,mr->mz', ovlp,
#                numpy.einsum('pr,mp->mr', tda_eig_vec_ao, 0.5*fock_RHS_all.alpha_to_numpy(1).T))
#LR_2pdm_F2 =  numpy.einsum('mr,rz->mz', ovlp,
#                numpy.einsum('rp,zp->rz', tda_eig_vec_ao, 0.5*fock_RHS_all.alpha_to_numpy(1)))

MO_rhs_2pdm = numpy.einsum('mi,ma->ia', mo_occ, 
                                 numpy.einsum('za,mz->ma', mo_vir,  
                                              (numpy.einsum('mr,rz->mz', ovlp,
                                                            numpy.einsum('rp,zp->rz',
                                                                         tda_eig_vec_ao, 
                                                                         0.5*fock_RHS_all.alpha_to_numpy(1))))
                                              
                                              -(numpy.einsum('rz,mr->mz', ovlp,
                                                             numpy.einsum('pr,mp->mr',
                                                                          tda_eig_vec_ao,
                                                                          0.5*fock_RHS_all.alpha_to_numpy(1).T))))
                          )

print(MO_rhs_2pdm)

[[ 0.0193612 -0.0000185]
 [ 0.0193312 -0.0000184]
 [-0.0001742  0.0000002]
 [ 0.0580351 -0.0000554]
 [ 0.        -0.       ]]


In [32]:
RHS = MO_rhs_1pdm+MO_rhs_2pdm

## Left-hand side
### Initial guess for Lagrange multipliers $\boldsymbol{\lambda}$

\begin{equation}
    \lambda_{ia}^{(0)} = \frac{R_{ia}}{\varepsilon_i - \varepsilon_a}
\end{equation}

In [39]:
#Construct Initial Guess: lambda_guess=RHS/(epsilon_i-epsilon_a)
orb_ene = scfdrv.scf_tensors['E']
eocc = orb_ene[:nocc]
evir = orb_ene[nocc:]
eov = -evir + eocc.reshape(-1, 1)
#print(eov)

lambda_guess = RHS / eov

print(lambda_guess)

[[-0.0000493 -0.0000003]
 [ 0.0339298  0.0000256]
 [-0.0002304  0.0766874]
 [ 0.0313636 -0.0000823]
 [ 0.         0.       ]]


### Contraction of the initial guess for $\boldsymbol{\lambda}$ with two-electron integrals

using $p=j, q=b$ as well as $p=b, q=j$ with the condition $\lambda_{pq} = \lambda_{qp}$ and assuming real orbitals, the equation is obtained as
\begin{align}
-\sum_{pq} \lambda_{pq} \big( [pq|ai] - [pi|aq] \big)
    &= -\sum_{\varphi \theta} C_{\varphi i} C_{\theta a} \sum_{\mu \nu}
        \lambda_{\mu \nu} \big[ 4 (\mu \nu | \varphi \theta)
        - (\mu \theta | \varphi \nu) - (\nu \theta | \varphi \mu) \big] \\
    %+ (\varepsilon_a - \varepsilon_i ) \gamma_{ia}
    &= - \sum_{\varphi \theta} \big[ C_{\varphi i} C_{\theta a} + C_{\theta i} C_{\varphi a} \big]
        \sum_{\mu \nu} \lambda_{\mu \nu} \big[ 2 (\mu \nu | \varphi \theta) - (\mu \theta | \varphi \nu) \big]
\end{align}

#### Full LHS 
\begin{equation}
   -\sum_{pq} \lambda_{pq} \big( [pq|ai] - [pi|aq] \big) + \lambda_{ia} ( \varepsilon_i - \varepsilon_a )
\end{equation}

## Solution via Conjugate Gradient

To solve: $\mathbf{A} \mathbf{x} = \mathbf{b}$

However, we do not "know" the matrix $\mathbf{A}$, only the matrix-vector product $\mathbf{Ax}$.
Thus, define it as a linear operator with a vector $\mathbf{x}$ as an argument.

### Defining a linear operator

In [45]:
lambda_guess_ao = numpy.matmul(mo_occ, numpy.matmul(lambda_guess, mo_vir.T))
    
#Create AODensityMatrix object from lambda in AO
ao_density_lambda_guess = AODensityMatrix([lambda_guess_ao], denmat.rest)
fock_lambda = AOFockMatrix(ao_density_lambda_guess)
fock_lambda.set_fock_type(fock_flag, 0)

#Matrix-vector product routine for conjugate gradient:
def OrbRsp_LHS_mvp(v):
    """Function to carry out matrix multiplication
    of Lagrange multipier vector with orbital Hessian
    matrix, using AODensityMatrix, AOFockMatrix
    
    v: lambda at current iteration"""
    
    #lambda_mo = v.reshape(nocc,nvir)
    
    # Transform to AO
    lambda_ao = numpy.matmul(mo_occ, numpy.matmul(v.reshape(nocc,nvir), mo_vir.T))
    
    #Create AODensityMatrix object from lambda in AO
    ao_density_lambda = AODensityMatrix([lambda_ao], denmat.rest)
    
    #Create a Fock Matrix Object (initialized with zeros)    
    eri_drv.compute(fock_lambda, ao_density_lambda, molecule, basis, screening) 
    
    #Transform to MO basis (symmetrized w.r.t. occ. and virt.) and add diagonal part
    lambda_mo = (-( numpy.matmul(mo_occ.T, numpy.matmul(fock_lambda.alpha_to_numpy(0), mo_vir)) 
                + numpy.matmul(mo_vir.T, numpy.matmul(fock_lambda.alpha_to_numpy(0), mo_occ)).T )
                +v.reshape(nocc,nvir) * eov) 
    
    return lambda_mo.reshape(nocc*nvir)

In [46]:
#OrbRsp_LHS_mvp(lambda_guess.reshape(nocc*nvir))
A = linalg.LinearOperator((nocc*nvir,nocc*nvir), matvec=OrbRsp_LHS_mvp)
solution, w = linalg.cg(A=A, b=RHS.reshape(nocc*nvir), x0=lambda_guess.reshape(nocc*nvir), tol=1e-8, maxiter=25)
print(solution.reshape(nocc,nvir))

[[-0.0001931 -0.0000005]
 [ 0.0302105  0.0000041]
 [-0.0002684  0.0793038]
 [ 0.0182954 -0.0001408]
 [-0.         0.       ]]
